<a href="https://colab.research.google.com/github/SadiyaMayat/NLP/blob/main/Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import necessary libraries.
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Path to your dataset
file_path = 'IMDB Dataset.csv'

# Load the dataset
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
# Initialize the lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to clean and preprocess text
def preprocess_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and lemmatize the words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Join the words back into a single string
    return ' '.join(words)

# Apply the preprocessing function to the review column
df['cleaned_review'] = df['review'].apply(preprocess_text)

# Convert sentiment column to numerical labels
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
df.head()

,review,sentiment,cleaned_review,label
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching 1 oz episode y...,1
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...,1
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,1
3,Basically there's a family where a little boy ...,negative,basically there family little boy jake think t...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...,1


In [ ]:
# Vocabulary size and max sentence length
voc_size = 10000
sent_length = 20

# Convert text to one-hot encoded sequences
corpus = df['cleaned_review'].tolist()
onehot_repr = [one_hot(words, voc_size) for words in corpus]

# Pad the sequences
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

In [ ]:
# Prepare training and testing data
X = np.array(embedded_docs)
y = np.array(df['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Model building (Use Bidirectional LSTM)
embedding_vector_features = 100
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 100)           1000000   
                                                                 
 dropout_3 (Dropout)         (None, 20, 100)           0         
                                                                 
 bidirectional (Bidirection  (None, 200)               160800    
 al)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 1161001 (4.43 MB)
Trainable params: 1161001 (4.43 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
# Model training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64)

Epoch 1/20
547/547 [==============================] - 67s 115ms/step - loss: 0.5091 - accuracy: 0.7428 - val_loss: 0.4543 - val_accuracy: 0.7824
Epoch 2/20
547/547 [==============================] - 60s 110ms/step - loss: 0.4002 - accuracy: 0.8192 - val_loss: 0.4647 - val_accuracy: 0.7787
Epoch 3/20
547/547 [==============================] - 62s 113ms/step - loss: 0.3468 - accuracy: 0.8476 - val_loss: 0.4927 - val_accuracy: 0.7676
Epoch 4/20
547/547 [==============================] - 61s 111ms/step - loss: 0.2917 - accuracy: 0.8771 - val_loss: 0.5725 - val_accuracy: 0.7616
Epoch 5/20
547/547 [==============================] - 60s 110ms/step - loss: 0.2438 - accuracy: 0.8985 - val_loss: 0.6092 - val_accuracy: 0.7608
Epoch 6/20
547/547 [==============================] - 62s 114ms/step - loss: 0.1892 - accuracy: 0.9229 - val_loss: 0.7324 - val_accuracy: 0.7531
Epoch 7/20
547/547 [==============================] - 65s 119ms/step - loss: 0.1493 - accuracy: 0.9403 - val_loss: 0.7723 - val_ac

In [ ]:
# Model evaluation
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

469/469 [==============================] - 9s 17ms/step
Confusion Matrix:
[[5470 1941]
 [1918 5671]]
Accuracy Score: 0.7427333333333334
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.74      0.74      7411
           1       0.75      0.75      0.75      7589

    accuracy                           0.74     15000
   macro avg       0.74      0.74      0.74     15000
weighted avg       0.74      0.74      0.74     15000

